In [ ]:
import six
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

!pip install git+https://github.com/google/qkeras

# Tensorflow Colab file with some modifications

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from qkeras import *

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
#train_labels, test_labels = get_one_hot(train_labels, 10), get_one_hot(test_labels, 10)#

train_images = train_images.reshape(train_images.shape + (1,)).astype("float32")
test_images = test_images.reshape(test_images.shape + (1,)).astype("float32")

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from qkeras import *

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

In [ ]:
intBits = 1
precision = 17

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='sigmoid'))
#model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c1'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
#model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c2'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
#model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c3'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='sigmoid'))
#model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        name='d1'))
#model.add(QActivation('quantized_relu({}, {})'.format(precision, intBits)))
model.add(layers.Dense(10))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images[:5000], test_labels[:5000]))

In [ ]:
import qkeras
from qkeras import *
from qkeras.utils import *

In [ ]:
model_save_quantized_weights(model)
print ("Done")

In [ ]:
model.evaluate(test_images[5000:], test_labels[5000:])

In [ ]:
### Evaluation in a loop
intBits = 1
histories = []
mymodels = []
for precision in [32]:
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c1'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c2'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c3'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))
  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(Activation('sigmoid'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))
  model.add(layers.Dense(10))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  history = model.fit(train_images[:48000], train_labels[:48000], epochs= 20, 
                    validation_data=(train_images[48000:], train_labels[48000:]), verbose=False)
  
  model_save_quantized_weights(model)
  print ("Done")

  model.evaluate(test_images, test_labels)

  mymodels.append(model)
  histories.append(history)

In [ ]:
histories[4].history['val_accuracy']

# Now with CORDIC

In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(arr, precision, intBits, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  xarr = 1.2075*np.ones(shape=(len(arr), len(arr[0])))
  yarr = np.zeros(shape = (len(arr), len(arr[0])))

  global  lookup_arctanh

  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(arr)
    
    xchange = m*sigma*2**(-i)*yarr
    ychange = sigma*2**(-i)*xarr
    arrchange = sigma*lookup_arctanh[i-1]

    xarr -= xchange
    yarr += ychange
    arr -= arrchange
    
  ex = xarr + yarr

  #temp = quantized_bits(precision, 1, alpha=1)(1 + ex)
  return quantized_bits(precision,intBits, alpha=1)(ex/(1+ex))

In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(arr, precision, intBits, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  xarr = 1.2075*np.ones(shape=(len(arr), len(arr[0])))
  yarr = np.zeros(shape = (len(arr), len(arr[0])))
  arr = -1*arr
  global  lookup_arctanh

  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(arr)
    
    xchange = m*sigma*2**(-i)*yarr
    ychange = sigma*2**(-i)*xarr
    arrchange = sigma*lookup_arctanh[i-1]

    xarr -= xchange
    yarr += ychange
    arr -= arrchange
    
  ex = xarr + yarr

  #temp = quantized_bits(precision, 1, alpha=1)(1 + ex)
  return quantized_bits(precision,intBits, alpha=1)(1/(1+ex))

In [ ]:
tangents = 2**(-1*np.arange(1.0, 100.0, 1.0))
lookup_arctanh = np.arctanh(tangents)

def modifiedCordicTanh(x, iterations):
  '''
  Returns the quantized tanh of the supplied argument x
  '''
  

  global  lookup_arctanh
  current_vector = np.array([1.2075, 0])
  z = x

  for i in range(1, iterations+1):
    m = -1
    sigma = np.sign(z)
    x = current_vector[0]
    y = current_vector[1]
    xnew = x
    ynew = y
    xnew = xnew - m*sigma*2**(-i)*y
    ynew = ynew + sigma*2**(-i)*x
    z = z - sigma*lookup_arctanh[i-1]
    current_vector = [xnew, ynew]

  
  ex = current_vector[0] + current_vector[1]
  eminusx = current_vector[0] - current_vector[1]

  temp1 = quantized_bits(9,1,alpha=1)(ex - eminusx)
  temp2 = quantized_bits(9, 1, alpha=1)(ex + eminusx)

  return quantized_bits(9,1, alpha=1)(temp1/temp2)

In [ ]:
qmodel = 0 

def epicGeneratePredictions(indices, precision, intBits, iterations):
  global qmodel
  x_test = test_images
  from keras import backend as K
  get_sixth_layer_output = K.function([qmodel.layers[0].input],
                                    [qmodel.layers[6].output])
 
  layer6_output = get_sixth_layer_output([x_test[indices[0]: indices[1]]])[0]

  #layer6_output = layer6_output[indices[0]: indices[1]]

  layer7_output = modifiedCordicTanh(layer6_output, precision, intBits, iterations)

  input_shape = qmodel.layers[8].get_input_shape_at(0)
  layer_input = Input(shape=(64))
  x = layer_input
  x = qmodel.layers[8](x)
  qm4 = Model(layer_input, x)

  predictions = np.array(qm4.predict(layer7_output))

  #predictions = modifiedCordicTanh(predictions, precision, iterations)

  a = predictions
  return (a == a.max(axis=1)[:,None]).astype(int)

In [ ]:
def modelMaker(precision, intBits, epoch=10):
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c1'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c2'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c3'))

  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(Activation('sigmoid'))
  model.add(layers.Dense(10))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)

  history = model.fit(train_images[:48000], train_labels[:48000], epoch, 
                    validation_data=(train_images[48000:], train_labels[48000:]), verbose=False, callbacks=[callback])
  
  model_save_quantized_weights(model)
  return model

Epochs:

for 8bit: 5 + int(precision/4)

for 12 bit: 10 + "

for 16 bit: 

## 8bit

In [ ]:
# # 1/ 1 + e-x

for epoch in range(5, 15):
  qmodel = modelMaker(9, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 9,1, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

## 12 bit

In [ ]:
for epoch in range(3, 15):
  qmodel = modelMaker(13, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 13,1, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
qmodel = modelMaker(13, 1, 11)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([0, 10000], 13, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[0:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(10000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

## 16 bit

In [ ]:
for epoch in range(3, 20):
  qmodel = modelMaker(17, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 17,1, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
qmodel = modelMaker(17, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([0, 10000], 17, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[0:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(10000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

## 24 bit

In [ ]:
for epoch in range(3, 15):
  qmodel = modelMaker(24, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 24,1, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
for epoch in range(15, 20):
  qmodel = modelMaker(25, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 25,1, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

## 32 Bit

In [ ]:
for epoch in range(3, 20):
  qmodel = modelMaker(33, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 33,1, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  print (accuracy)

In [ ]:
qmodel = modelMaker(33, 1)
answerVectors = []
for iter in [2,3,4, 5, 6, 7, 8]:
  answerVectors.append(epicGeneratePredictions([0, 10000], 33, iter))
  print ("Done")

accuracy = []
y_test = get_one_hot(test_labels[0:10000], 10)
for i in answerVectors:
  correct = 0
  for j in range(10000):
    if (i[j] == y_test[j]).all():
      correct += 1
  accuracy.append(correct)

accuracy

In [ ]:
for epoch in range

In [ ]:
qmodel.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.1153 - accuracy: 0.9680


[0.11529628932476044, 0.9679999947547913]

In [ ]:
accuracies = []
for epoch in range(5, 20):

  qmodel = modelMaker(33, 1, epoch)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 33, iter))
    print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  accuracies.append(accuracy)


In [ ]:
accuracies

# Varying Binary Point

## 8 bit

In [ ]:
# Here we take 8bit sigmoid...and vary the binary point and determine the accuracy

In [ ]:
accuracies = []
for intBits in range(9):
  epochAccuracy = []
  for epoch in range(5, 20):
    qmodel = modelMaker(9, intBits, epoch)
    answerVectors = []
    for iter in [2,3,4, 5, 6, 7, 8]:
      answerVectors.append(epicGeneratePredictions([0, 10000], 9, intBits, iter))
      #print ("Done")

    accuracy = []
    y_test = get_one_hot(test_labels[0:10000], 10)
    for i in answerVectors:
      correct = 0
      for j in range(10000):
        if (i[j] == y_test[j]).all():
          correct += 1
      accuracy.append(correct)

    epochAccuracy.append(accuracy)
  epochAccuracy = np.array(epochAccuracy)
  accuracies.append(np.max(epochAccuracy, axis=0))

In [ ]:
accuracies

In [ ]:
accuracies

In [ ]:
accuracies

## 12 Bits

In [ ]:
accuracies = []
for intBits in range(13):
  epochAccuracy = []
  for epoch in range(5, 10):
    qmodel = modelMaker(13, intBits, epoch)
    answerVectors = []
    for iter in [2,3,4, 5, 6, 7, 8]:
      answerVectors.append(epicGeneratePredictions([0, 10000], 13, intBits, iter))
      #print ("Done")

    accuracy = []
    y_test = get_one_hot(test_labels[0:10000], 10)
    for i in answerVectors:
      correct = 0
      for j in range(10000):
        if (i[j] == y_test[j]).all():
          correct += 1
      accuracy.append(correct)

    epochAccuracy.append(accuracy)
  epochAccuracy = np.array(epochAccuracy)
  accuracies.append(np.max(epochAccuracy, axis=0))

In [ ]:
accuracies

In [ ]:
accuracies = []
for intBits in range(13):
  qmodel = modelMaker(13, intBits, 20)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 13, intBits, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  accuracies.append(accuracy)

In [ ]:
accuracies

## 16 Bit

In [ ]:
accuracies = []
for intBits in range(17):
  epochAccuracy = []
  for epoch in range(10, 18):
    qmodel = modelMaker(17, intBits, epoch)
    answerVectors = []
    for iter in [2,3,4, 5, 6, 7, 8]:
      answerVectors.append(epicGeneratePredictions([0, 10000], 17, intBits, iter))
      #print ("Done")

    accuracy = []
    y_test = get_one_hot(test_labels[0:10000], 10)
    for i in answerVectors:
      correct = 0
      for j in range(10000):
        if (i[j] == y_test[j]).all():
          correct += 1
      accuracy.append(correct)

    epochAccuracy.append(accuracy)
  epochAccuracy = np.array(epochAccuracy)
  accuracies.append(np.max(epochAccuracy, axis=0))

In [ ]:
accuracies

In [ ]:
accuracies

In [ ]:
accuracies

In [ ]:
accuracies = []
for intBits in range(17):
  qmodel = modelMaker(17, intBits, 25)
  answerVectors = []
  for iter in [2,3,4, 5, 6, 7, 8]:
    answerVectors.append(epicGeneratePredictions([0, 10000], 17, intBits, iter))
    #print ("Done")

  accuracy = []
  y_test = get_one_hot(test_labels[0:10000], 10)
  for i in answerVectors:
    correct = 0
    for j in range(10000):
      if (i[j] == y_test[j]).all():
        correct += 1
    accuracy.append(correct)

  accuracies.append(accuracy)

In [ ]:
accuracies

In [ ]:
accuracies 

# 24 bit accuracy values

## Tensorflow

In [ ]:
### Evaluation in a loop
precision = 25
intBits = 1
histories = []
mymodels = []
for epoch in range(3, 13):
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c1'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c2'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='sigmoid',
        name='c3'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))
  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(Activation('sigmoid'))
  model.add(QActivation('quantized_bits({}, {})'.format(precision, intBits)))
  model.add(layers.Dense(10))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  history = model.fit(train_images[:48000], train_labels[:48000], epochs= epoch, 
                    validation_data=(train_images[48000:], train_labels[48000:]), verbose=False)
  
  model_save_quantized_weights(model)
  print ("Done")

  model.evaluate(test_images, test_labels)

  mymodels.append(model)
  histories.append(history)